Discussed this assignment with Hyun In Park, Heeseung Hwang, and Sang Hoon Kim.  
This notebook was originally written and executed on Google colab, then opened via Jupyter for PDF translation.

**Preliminary setup:**  

As the task is overall highly memory-consuming and often crashes the session, I serialized key data at each checkpoint. If the session gets killed, I did not re-execute entirety of the code but simply loaded stored pickle files and continued on the leftover parts.

In [0]:
import nltk
import pickle
import numpy as np
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from collections import Counter

In [0]:
with open("/content/drive/My Drive/cmsc25025/wiki-text.txt", "r") as f:
  wikidata = f.read().replace("\n", " ")
  f.close()

wikidata = wikidata.lower().split()[:25000000]
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
cnt_wikidata = Counter(wikidata)

After filtering out stop_words and those appearing 100 times or less, I got the final vocabulary set with size of 14960.  
The corpus and vocabulary size (i.e. filtering threshold) are determined with the target of (i) having approx. 15000 words in the vocabulary set, and (ii) keep the corpus and vocabulary to the manageable size so that the session won't crash (often, at least).

In [0]:
wikidata = list(filter(lambda x: x not in stop_words, wikidata))
wikidata = list(filter(lambda x: cnt_wikidata[x] > 100, wikidata))

In [0]:
vocabulary = set(wikidata)

In [0]:
with open(r"/content/drive/My Drive/cmsc25025/saved_wikidata.p", "wb") as output_file:
  pickle.dump(wikidata, output_file)
  output_file.close()

In [0]:
with open(r"/content/drive/My Drive/cmsc25025/saved_vocab.p", "wb") as output_file:
  pickle.dump(vocabulary, output_file)
  output_file.close()

In [0]:
with open(r"/content/drive/My Drive/cmsc25025/saved_wikidata.p", "rb") as input_file:
  wikidata = pickle.load(input_file)
  input_file.close()

In [0]:
with open(r"/content/drive/My Drive/cmsc25025/saved_vocab.p", "rb") as input_file:
  vocabulary = pickle.load(input_file)
  input_file.close()

In [0]:
len(vocabulary)

14960

**Part (a):**  

In [0]:
#Computing positive samples
Sp_pair = []

for i in range(len(wikidata)):

  center = wikidata[i]
  context = wikidata[i-5:i] + wikidata[i+1:i+6]

  Sp_pair += [(center, word) for word in context]

with open(r"/content/drive/My Drive/cmsc25025/saved_sppair_all.p", "wb") as output_file:
  pickle.dump(Sp_pair, output_file)
  output_file.close()

In [0]:
#Saving positive samples
with open(r"/content/drive/My Drive/cmsc25025/saved_sppair_all.p", "rb") as input_file:
  Sp_pair = pickle.load(input_file)
  input_file.close()

In [0]:
spraw0 = [x[0] for x in Sp_pair]
spraw1 = [x[1] for x in Sp_pair]
Sp_raw = spraw0 + spraw1

In [0]:
cnt_Sp_pair = Counter(Sp_pair)
cnt_Sp_raw = Counter(Sp_raw)

In [0]:
def compute_pmi_row(i):
  '''
  Computes ith row of M
  '''
  wi = vocablist[i]
  N_wi = cnt_Sp_raw[wi]

  #treating the positive sample set with the unordered-ness/symmetricity of (w,c) in consideration
  row_numer = np.array([cnt_Sp_pair[(wi, wj)]+cnt_Sp_pair[(wj, wi)]+1 for wj in vocablist])
  row_denom = np.array([cnt_Sp_raw[wj] for wj in vocablist])
  row = row_numer / row_denom
  row = np.log(row * N_Sp / N_wi)

  return row

In [0]:
M = np.zeros((len(vocabulary), len(vocabulary)))
vocablist = list(vocabulary)
N_Sp = len(Sp_pair)

for i in range(len(vocabulary)):
  
  M[i, :] = compute_pmi_row(i)

**Part (b):**

In [0]:
#SVD
U, Sigma, Vt = svds(csr_matrix(M), k = 50)
Sigma = np.diag(Sigma)

**Part (c):**

In [0]:
W = np.matmul(U, Sigma ** 0.5) #embedding matrix

In [0]:
# Saves embedding matrix
with open(r"/content/drive/My Drive/cmsc25025/saved_embedding.p", "wb") as output_file:
  pickle.dump(W, output_file)
  output_file.close()

**Part (d):**

In [0]:
def get_n_closest(word, n, embedding, already_exists = True):
  '''
  word should be a string if it already exists = True, and
  a vector otherwise
  '''
  if already_exists:
    ind = vocablist.index(word)
    embedding = embedding - embedding[ind]
    distance = np.apply_along_axis(np.linalg.norm, -1, embedding)
    closest_ind = list(np.argsort(distance)[1:n+1])
  
  else:
    embedding = embedding - word
    distance = np.apply_along_axis(np.linalg.norm, -1, embedding)
    closest_ind = list(np.argsort(distance))[:n]

  return closest_ind

In [0]:
examples = {"physics": [], "republican": [], "einstein": [], "algebra": [], "fish": []}

for word in examples:

  closest = get_n_closest(word, 5, W)

  for ind in closest:
    examples[word].append(vocablist[ind])
  
for word in examples:
  print("{:s} is most similar to: {:s}, {:s}, {:s}, {:s}, {:s}".format(
      word, examples[word][0], examples[word][1], examples[word][2], 
      examples[word][3], examples[word][4]))

physics is most similar to: mechanics, quantum, einstein, relativity, fields
republican is most similar to: democrats, presidential, presidency, electoral, senator
einstein is most similar to: relativity, newton, experiment, maxwell, physicists
algebra is most similar to: algebraic, finite, theorem, calculus, spaces
fish is most similar to: fruit, wild, meat, trees, plant


For physics, republican, einstein, and algebra, all top 5 similar words appear to be very relevant to the respective vocabulary.  
For example, "quantum mechanics" is a field of "physics," "relativity" is a research achievement by "einstein," who is a famous "physicist(s)" just like "newton" or "maxwell." "Repbulican" will compete with "democrats" for the "presidency," and "algebra" will involve "theorems" about (say, vector) "spaces."  
On the other hand, similar words to "fish" seem to be selected from a broader domain. While all of 5 words have connections to "fish" in some sense, "fruit" and "meat" appear to consider the food-aspect of fish, while "wild," "trees," and "plant" consider the animal-aspect of fish (within the broader scope of biology/ecology/nature, etc.).

**Part (e):**

3 analogies I used are:

einstein:scientist = clinton: ?  
left:right = democrat: ?    
love:hate = peace: ?  

The natural answers I expect by common sense are: "president," "republican," and "war."

In [0]:
analogies = [("einstein", "scientist", "clinton"), ("left", "right", "democrat"), ("love", "hate", "peace")]
guess = []

for case in analogies:

  v1_ind, v2_ind, v3_ind = vocablist.index(case[0]), vocablist.index(case[1]), vocablist.index(case[2])
  v = W[v2_ind, :] - W[v1_ind, :] + W[v3_ind, :]

  guess_ind = get_n_closest(v, 1, W, False)
  guess.append(vocablist[guess_ind[0]])

In [0]:
print("Analogy guess made by the algorithm\n")

for i in range(3):
  #print("{:s} : {:s} = {:s} : {:s}".format(analogies[i][0], analogies[i][1], analogies[i],[2], guess[i]))
  print(analogies[i][0], " : ", analogies[i][1], " = ", analogies[i][2], " : ", guess[i])

Analogy guess made by the algorithm

einstein  :  scientist  =  clinton  :  kennedy
left  :  right  =  democrat  :  amendment
love  :  hate  =  peace  :  negotiations


Answers for all three analogies were different from my expectation, but those answers by the algorithm still seem to be grasping some sense of context / relationship between words.  
For example, based on enstein-scientist (person-occupation) pair, the program guessed "kennedy"--another president (i.e. same occupation) from the same party--for "clinton."  
Based on the left-right analogy (antonyms), the software brought "amendment" for "democrat." It is not "republican" as expected, but still in the context of politics. (I'm not sure if republicans like amendments more than democrats do, though.)  
Finally, based on love-hate analogy (antonyms again), I expected "war" while the algorithm guessed "negotiations" for "peace." I'd say negotiation is definitely relevant to war and thus the program is not really missing the context, but whether or not it is an antonym of "peace" could be controversial.  

If it was given more data (at the cost of computing time) or the original corpus was pre-processed better (in terms of text processing, e.g. regex, etc.), the algorithm might have performed better on these analogies.